## EPC Extent

In [10]:
help(gdal.DEMProcessingOptions)

Help on function DEMProcessingOptions in module osgeo.gdal:

DEMProcessingOptions(options=None, colorFilename=None, format=None, creationOptions=None, computeEdges=False, alg='Horn', band=1, zFactor=None, scale=None, azimuth=None, altitude=None, combined=False, multiDirectional=False, igor=False, slopeFormat=None, trigonometric=False, zeroForFlat=False, addAlpha=None, callback=None, callback_data=None)
    Create a DEMProcessingOptions() object that can be passed to gdal.DEMProcessing()
    Keyword arguments are :
      options --- can be be an array of strings, a string or let empty and filled from other keywords.
      colorFilename --- (mandatory for "color-relief") name of file that contains palette definition for the "color-relief" processing.
      format --- output format ("GTiff", etc...)
      creationOptions --- list of creation options
      computeEdges --- whether to compute values at raster edges.
      alg --- 'ZevenbergenThorne' or 'Horn'
      band --- source band numb

In [6]:
import os
import pdal
import gdal
from datetime import datetime
import json
import rasterio as rio
from joblib import Parallel, delayed

json_pipe_base = """ { "pipeline": [ %s ] } """

gdal_writer = """
    "%s",
    {
        "type":"writers.gdal",
        "filename": "%s",
        "resolution": %s,
        "window_size": %s,
        "nodata": 0,
        "output_type": "idw",
        "gdalopts":"TILED=YES, COMPRESS=LZW"
    }
"""

def createDSM(file_dict, res, overwrite=False):
    infile = file_dict["in_file"]
    outfile = file_dict["out_file"]
    
    if os.path.exists(outfile) and not overwrite:
        return
  
    json_dsm = gdal_writer % (infile.replace("\\","/"), outfile.replace("\\","/"), res, max(res * 2,10))#, ox, oy)#, raswidth, rasheight)
    json_dsm = json_pipe_base % json_dsm
    
    pipeline = pdal.Pipeline(json_dsm)
    pipeline.loglevel = 8

    start = datetime.now()
    pipeline.execute()
    end = datetime.now()

    print(f"{datetime.now()}\t-\tFinished with {outfile} - {end-start} elapsed")


resolution = 0.8/0.3048

ortho_folder = r"../EPCExtent_30cm/Orthos/"
laz_folder = r"../EPCExtent_30cm/Elevation_80cmNPS/RGB/Full"
dsm_folder = f"../EPCExtent_30cm/Elevation_80cmNPS/DSM_{resolution}"
os.makedirs(dsm_folder, exist_ok=True)

infiles = []
for f in os.listdir(laz_folder):
    infile = os.path.join(laz_folder, f)
    outfile = os.path.join(dsm_folder, f.replace(".laz", f"_DSM{round(resolution*.3048*100,0)}cm.tif"))
    if f.endswith(".laz"):# and not os.path.exists(outfile):
        files = {}
        files["in_file"] = infile
        files["out_file"] = outfile
        infiles.append(files)

print("\nBeginning DSM Generations for {} QQuads\n".format(len(infiles)))
Parallel(n_jobs=5, max_nbytes=None, verbose=30)(delayed(createDSM)(f, resolution) for f in infiles)
for f in infiles:
    hour = int(datetime.strftime(datetime.now(), '%H'))
    #while hour > 9 and hour < 17:
    #    print("Pausing creation for 1 hours...")
    #    time.sleep(60*60)
    #    hour = int(datetime.strftime(datetime.now(), '%H'))
        
    createDSM(f)
    
print("\n\n...FINISHED\n\n")


Beginning DSM Generations for 327 QQuads



[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   4 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   5 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   7 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  11 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  12 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  13 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  14 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    0.9s
[Parallel(

TypeError: createDSM() missing 1 required positional argument: 'res'

## URBAN EXTENT

In [3]:
%load_ext chime
import os
import pdal
import gdal
from datetime import datetime
import json
import rasterio as rio
from joblib import Parallel, delayed
import geopandas as gpd
import chime

The chime extension is already loaded. To reload it, use:
  %reload_ext chime


PROJ: proj_create_from_database: Cannot find proj.db


In [4]:
ugt_tindex_loc = r"E:/LAZIndex2019_UGTRGBFull.gpkg"
ugt_tindex = gpd.read_file(ugt_tindex_loc)#.to_crs("epsg:2868')

ortho_5ksubindex = r"M:\PAG2019\EPCExtent_30cm\Ortho_5kSubIndex.gpkg"
subIndex5k = gpd.read_file(ortho_5ksubindex)
#subIndex5k.head()

dsm40cm_loc = r"..\UrbanExtent_15cm\Elevation_40cmNPS\DSM40cm"

ugt_res = 0.4/0.3048

In [5]:
json_pipe_base = """{ "pipeline":
[ %s ]
}"""

json_reindex_base = """
    %s
    {
        "type":"filters.merge"
    },
    {
        "type":"filters.crop",
        "polygon":"%s",
        "a_srs":"EPSG:2868"
    }"""

gdal_writer = """
    %s,
    {
        "type":"writers.gdal",
        "filename": "%s",
        "resolution": %s,
        "window_size": %s,
        "nodata": 0,
        "output_type": "idw",
        "gdalopts":"TILED=YES, COMPRESS=LZW, "
    }
"""

def lazToDSM(row):    
    #for i, row in subIndex5k.iterrows():
    geom = row.geometry
    outfile_name = f"{row.path}_{row.row}_DSM40cm.tif"
    ofile = os.path.join(dsm40cm_loc, outfile_name)
    print(f"Starting {ofile} at {datetime.now()}")
    
    if not os.path.exists(ofile):
        geom_wkt = str(geom.wkt)

        lidar_tiles = ugt_tindex[ugt_tindex.geometry.intersects(geom)]
        if len(lidar_tiles) == 0:
            #continue
            return None

        files = '"' + '",\n"'.join([os.path.abspath(file).replace("\\","/") for file in lidar_tiles.location.values]) + '",'


        json_reindex = json_reindex_base % (files, geom_wkt)#, pdal_bounds_min, pdal_bounds_max
        create_dsm = gdal_writer % (json_reindex, ofile.replace("\\","/"), ugt_res, max(ugt_res * 2, 5))


        json_pipeline = json_pipe_base % create_dsm
        #print(json_pipeline)

        start = datetime.now()
        pipeline = pdal.Pipeline(json_pipeline)
        pipeline.loglevel = 8
        pipeline.execute()
        end = datetime.now()
        print(f"\tFinished with {ofile} - {end-start} elapsed")

        return ofile

In [12]:
def getToDo(df):
    existing_files = os.listdir(dsm40cm_loc)
    df["outfile"] = df.apply(lambda row: f"{row.path}_{row.row}_DSM40cm.tif", axis=1)
    df= df[~df.outfile.isin(existing_files)]
    del df["outfile"]
    return df

In [13]:
toDo = getToDo(subIndex5k)
completed_ugt = Parallel(n_jobs=5, max_nbytes=None, verbose=5)(delayed(lazToDSM)(r) for i,r in toDo.iterrows())
%chime

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:  3.1min
C:\Users\BenJames\anaconda3\envs\geospatial\lib\site-packages\joblib\externals\loky\process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed: 18.7min
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed: 40.6min
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed: 72.7min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 121.2min
[Parallel(n_jobs=5)]: Done 638 tasks      | elapsed: 173.6min
[Parallel(n_jobs=5)]: Done 790 out of 790 | elapsed: 201.9min finished


In [ ]:
def createDSM(file_dict, res, overwrite=False):
    infile = file_dict["in_file"]
    outfile = file_dict["out_file"]
    
    if os.path.exists(outfile) and not overwrite:
        return
  
    json_dsm = gdal_writer % (infile.replace("\\","/"), outfile.replace("\\","/"), res, max(res * 2,10))#, ox, oy)#, raswidth, rasheight)
    json_dsm = json_pipe_base % json_dsm
    
    pipeline = pdal.Pipeline(json_dsm)
    pipeline.loglevel = 8

    start = datetime.now()
    pipeline.execute()
    end = datetime.now()

    print(f"{datetime.now()}\t-\tFinished with {outfile} - {end-start} elapsed")
    
    
def mergeToTiles(row, out_dir, tindex):
    name = row.THEMENAME
    print(f"Starting {name} at {datetime.now()}")
    ofile = os.path.join(out_dir, name + ".laz")
    if not os.path.exists(ofile):
        try:
            geom = row.geometry
            buff_geom = box(*geom.buffer(100).bounds)
            geom_wkt = str(buff_geom.wkt)

            lidar_tiles = tindex[tindex.geometry.intersects(buff_geom)]
            files = ''
            for i in lidar_tiles.location.values:
                files += '"' + os.path.abspath(i).replace('\\','/') + '", '
            
            files = files[:-1]

            query = ""
            for idx, r in lidar_tiles.iterrows():
                query += f"ogc_fid = {idx} or "
            query = query[:-4]

            json_reindex = json_reindex_base % (files, geom_wkt, ofile.replace('\\','/'), "EPSG:2868")#, pdal_bounds_min, pdal_bounds_max
            json_reindex = json_pipe_base % json_reindex
            
            start = datetime.now()
            pipeline = pdal.Pipeline(json_reindex)
            pipeline.loglevel = 8
            pipeline.execute()
            end = datetime.now()
            #print(f"\tFinished with {row.THEMENAME} - {end-start} elapsed")
            
            return json_reindex
        
        except MemoryError:
            print(Back.RED + Fore.WHITE + f"Failed for {name}. \n{MemoryError}")
            return None

In [ ]:
import os
import pdal
import gdal
from datetime import datetime
import json
import rasterio as rio
from joblib import Parallel, delayed
import time


ortho_folder = r"P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Orthos"
laz_folder = r"P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Elevation\RGB\Full_retiled"
dsm_folder = r"P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Elevation\DSM_2ft"
os.makedirs(dsm_folder, exist_ok=True)

resolution = 0.8/0.3048

infiles = []
for f in os.listdir(laz_folder):
    infile = os.path.join(laz_folder, f)
    outfile = os.path.join(dsm_folder, f.replace(".laz", "_DSM.tif"))
    if f.endswith(".laz") and not os.path.exists(outfile):
        files = {}
        files["in_file"] = infile
        files["out_file"] = outfile
        infiles.append(files)  

print("\nBeginning DSM Generations for {} QQuads\n".format(len(infiles)))
Parallel(n_jobs=5, max_nbytes=None, verbose=5)(delayed(createDSM)(f, resolution) for f in infiles)

for f in infiles:
    #if "E0980_N510_1" in f['out_file']:
    hour = int(datetime.strftime(datetime.now(), '%H'))
    """while hour > 9 and hour < 17:
        print("Pausing creation for 1 hours...")
        time.sleep(60*60)
        hour = int(datetime.strftime(datetime.now(), '%H'))"""
        
    createDSM(f)
    
print("\n\n...FINISHED\n\n")


Beginning DSM Generations for 217 QQuads

2019-12-17 09:57:24.591319	-	Finished with P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Elevation\DSM_2ft\E0920_N450_3_DSM.tif - 0:01:15.189486 elapsed
